In [1]:
!pip install transformers
!pip install gradio 
!pip install transformers-interpret
!pip install shap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 46.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 108.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 27.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 88.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.7/286.7 kB 35.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 8.5 MB/s eta 0:00:0

In [2]:
# Import required libraries
import numpy as np
import torch
from transformers_interpret import SequenceClassificationExplainer
import pandas as pd
import scipy as sp
import shap
import torch
import json
from transformers import (AutoModelForSequenceClassification, AutoTokenizer, TextClassificationPipeline)


def load_model(model_path):
    # Load the tokenizer and model from the "roberta-base" pre-trained model
    tokenizer = AutoTokenizer.from_pretrained("roberta-base",use_fast=True)
    model = AutoModelForSequenceClassification.from_pretrained("roberta-base").cuda()
    # Load the saved state dict of the fine-tuned model
    model.load_state_dict(torch.load(model_path))

    return tokenizer, model


def preprocess_text(tokenizer, input_text, max_length):
    # Tokenize the input text using the tokenizer
    inputs = tokenizer.encode_plus(
        input_text,
        add_special_tokens=True,
        return_tensors="pt",
        max_length=max_length,
        truncation=True,
    )

    # Get the input_ids and attention_mask tensors
    return inputs["input_ids"].cuda(), inputs["attention_mask"].cuda()


def get_prediction(model, input_ids, attention_mask):
    # Get the predicted label using the input_ids and attention_mask
    outputs = model(input_ids, attention_mask=attention_mask)
    score = outputs.logits.detach().cpu().numpy()
    predicted_label = np.argmax(score)
    return predicted_label

def fn(input):
  input_ids, attention_mask = preprocess_text(
                tokenizer, input, max_length=512
            )
  predicted_label = get_prediction(model, input_ids, attention_mask)
  labels = ["Negative","Positive"]
  #explanation1
  cls_explainer = SequenceClassificationExplainer(
    model, tokenizer, custom_labels = labels)
  cls_explainer(input)
  cls_explainer.visualize("text2.html")
  file=open("text2.html")
  explaination1 = ''.join(file.readlines())

  # SHAP
  device = "cuda:0"
  pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, device = device, return_all_scores=True,truncation=True)
  pmodel = shap.models.TransformersPipeline(pipe, rescale_to_logits=True)
  explainer = shap.Explainer(pmodel,output_names=labels)
  shap_values = explainer([input])
  file = open('temp.html','w')
  file.write(shap.plots.text(shap_values[:,:,1], display=False))
  file.close
  file=open("temp.html")
  explaination_shap = ''.join(file.readlines())
  


  if predicted_label == 1:
    return "Positive",explaination_shap,explaination1,
  else:
    return "Negative",explaination_shap,explaination1



# Load the fine-tuned model from the saved state dict
model_path = "/content/drive/MyDrive/DDA4210project/Roberta_model.pt"
tokenizer, model = load_model(model_path)




Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifie

In [4]:
import gradio as gr   
demo = gr.Interface(fn=fn, inputs=gr.Textbox(lines=2, placeholder="Enter sentence Here..."), 
                    outputs=[gr.Textbox(label='prediction'), gr.HTML(label = 'prediction1'), gr.HTML(label = 'prediction2')])
demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://75359cead8713e3797.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


In [ ]:
import numpy as np
import gradio as gr



with gr.Blocks() as demo:
    gr.Markdown("Explainable sentiment analysis")
    with gr.Tab("Roberta Sentiment Analysis"):
        text_input = gr.Textbox(lines=2, placeholder="Enter sentence Here...")
        text_output = gr.Textbox()
        text_button = gr.Button("Flip")
    with gr.Tab("Explanation"):
        gr.HTML(label = "Explanation1")
        gr.HTML(label = "Explanation1")
        

    
    text_button.click(fn=fn, inputs=input, outputs=text_output)
    image_button.click(fn=fn, outputs=image_output)

demo.launch()